In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from PIL import Image

In [ ]:
train_path = '/stress/KDEF/KDEF/Train'
valid_path = '/stress/KDEF/KDEF/Test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(256, 256), classes=['NoStress', 'Stress'], batch_size=64)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path, target_size=(256, 256), classes=['NoStress', 'Stress'], batch_size=64)

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
def PlotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
PlotImages(imgs)
print(labels)

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dense(units=2, activation='softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='BinaryCrossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, class_weight={0: 1.0, 1: 1.0}, epochs=5, verbose=1)

In [ ]:
def preprocess_image(img_path, target_size=(256, 256)):
    img = Image.open(img_path)
    img = img.resize(target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    return img_array

In [ ]:
def predict_image_class(img_path, model):
    preprocessed_img = preprocess_image(img_path)
    PlotImages(preprocessed_img)
    predictions = model.predict(preprocessed_img)
    return predictions

In [ ]:
img_path = 'image.jpg'
predicted_class = predict_image_class(img_path, model)
print(f'model is this confident the image does not show stress: {predicted_class[0][0]} and this confident the image shows stress: {predicted_class[0][1]}')

In [ ]:
#model.save('stressed.h5')